In [73]:
import dask.dataframe as dd
import pandas as pd
from datetime import datetime
import numpy as np
from dask import array as da

In [38]:
train = pd.read_csv('./data_train.csv')
train.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0
3,3,3535012,5.0,1535922000,0.0
4,4,1693214,1.0,1535922000,0.0


In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831653 entries, 0 to 831652
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  831653 non-null  int64  
 1   id          831653 non-null  int64  
 2   vas_id      831653 non-null  float64
 3   buy_time    831653 non-null  int64  
 4   target      831653 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 31.7 MB


In [40]:
type(train)

pandas.core.frame.DataFrame

In [62]:
def rm_un_and_chen_time(data: pd.core.frame.DataFrame):
    data.drop(['Unnamed: 0'], axis=1, inplace=True)
    data['buy_time'] = data['buy_time'].apply(lambda x: datetime.fromtimestamp(x))
    return data

In [42]:
train = rm_un_and_chen_time(train)
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831653 entries, 0 to 831652
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        831653 non-null  int64         
 1   vas_id    831653 non-null  float64       
 2   buy_time  831653 non-null  datetime64[ns]
 3   target    831653 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 25.4 MB


,id,vas_id,buy_time,target
0,540968,8.0,2018-09-17,0.0
1,1454121,4.0,2018-07-16,0.0
2,2458816,1.0,2018-08-13,0.0
3,3535012,5.0,2018-09-03,0.0
4,1693214,1.0,2018-09-03,0.0


In [64]:
test = pd.read_csv('data_test.csv')
test.head()

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400
3,3,1343255,5.0,1547413200
4,4,1277040,2.0,1546808400


In [65]:
test = rm_un_and_chen_time(test)
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71231 entries, 0 to 71230
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        71231 non-null  int64         
 1   vas_id    71231 non-null  float64       
 2   buy_time  71231 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.6 MB


,id,vas_id,buy_time
0,3130519,2.0,2019-01-21
1,2000860,4.0,2019-01-21
2,1099444,2.0,2019-01-07
3,1343255,5.0,2019-01-14
4,1277040,2.0,2019-01-07


In [58]:
dask_fu = dd.read_csv('./features.csv', sep='\t')

In [70]:
%%time 
dask_fu.head()

CPU times: user 20.8 s, sys: 4.29 s, total: 25.1 s
Wall time: 7.2 s


,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,0,2013026,1531688400,18.910029,46.980888,4.969214,-1.386798,3.791754,-14.01179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-301.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
1,1,2014722,1539550800,36.690029,152.400888,448.069214,563.833202,463.841754,568.99821,-16.08618,...,-891.373846,-544.770792,-20.996269,48.369552,80.252276,-13.832889,-0.694428,-1.175933,-0.45614,0.0
2,2,2015199,1545598800,-67.019971,157.050888,-63.180786,178.103202,-68.598246,156.99821,3.51382,...,-977.373846,-613.770792,-12.996269,-37.630448,10829.252276,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3,2021765,1534107600,7.010029,150.200888,-6.930786,216.213202,76.621754,351.84821,-16.08618,...,-973.373846,-613.770792,-23.996269,-37.630448,-205.747724,-24.832889,-0.694428,-11.175933,-0.45614,1.0
4,4,2027465,1533502800,-90.439971,134.220888,-104.380786,153.643202,-109.798246,132.53821,-16.08618,...,1643.626154,2007.229208,206.003731,-21.630448,6667.252276,92.167111,-0.694428,49.824067,47.54386,0.0


In [80]:


def truncate_features(df_features, train, test):
    user_ids = np.unique(train['id'].append(test['id']))
    df_features = df_features[df_features['id'].isin(user_ids)]

    cols = df_features.columns
    corr_threshold = 0.5
    corr = df_features[cols].corr().abs().values
    corr = corr.compute_chunk_sizes()
    corr = da.triu(corr)
    out = (corr != 1) & (corr > corr_threshold)
    out = out.compute()
    cols_to_remove = []
    for o in out:
      cols_to_remove += cols[o].to_list()
    cols_to_remove = list(set(cols_to_remove))
    if 'id' in cols_to_remove:
      cols_to_remove = cols_to_remove.drop(['id'], axis=1)
    df_features = df_features.drop(cols_to_remove, axis=1)

    if 'Unnamed: 0' in df_features:
        df_features = df_features.drop(columns='Unnamed: 0')
    df_features = df_features.compute()

    df_nunique = df_features.apply(lambda x: x.nunique(dropna=False))
    const = df_nunique[df_nunique == 1].index.tolist()
    df_features = df_features.drop(columns = const)
    return df_features



In [81]:
%%time 
features = truncate_features(dask_fu, train, test)
features.head()

AttributeError: 'list' object has no attribute 'drop'

In [77]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 902389 entries, 13 to 12421
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   id      902389 non-null  int64
dtypes: int64(1)
memory usage: 13.8 MB
